In [1]:
from config import spotify_client_ID, spotify_client_secret, my_spotify_uri, weather_api, my_redirect_url, username, scope
from spotipy.oauth2 import SpotifyClientCredentials
from json.decoder import JSONDecodeError

import datetime 
import openweathermapy.core as owm
import requests
import spotipy
import pprint
import sys
import re
import os
import pandas as pd
import spotipy.util as util
import json
import billboard
import time

In [2]:
year = datetime.datetime.today().year
years = list(range(year, year - 59, -1))

summer_charts = []
winter_charts =[]

for year in years:
    summer_chart = billboard.ChartData(f'hot-100/{year}-06-20')
    winter_chart = billboard.ChartData(f'hot-100/{year}-12-20')

    summer_charts.append(summer_chart)
    winter_charts.append(winter_chart)
    

In [4]:
summer_songs = []
winter_songs = []
summer_artists = []
winter_artists = []

for chart in summer_charts:
    for entry in chart:
        entry = str(entry)
        temp = re.split(r"(' by)", entry)
        song = temp[0].replace("'","")
        summer_songs.append(song)
        singer = temp[2]
        summer_artists.append(singer)

for chart in winter_charts:
    for entry in chart:
        entry = str(entry)
        temp = re.split(r"(' by)", entry)
        song = temp[0].replace("'","")
        winter_songs.append(song)
        singer = temp[2]
        winter_artists.append(singer)

In [5]:
print(summer_songs[0])
print(summer_artists[0])
print(len(summer_songs))
print(winter_songs[0])
print(winter_artists[0])
print(len(winter_songs))

Despacito
 Luis Fonsi & Daddy Yankee Featuring Justin Bieber
5800
Perfect
 Ed Sheeran
5800


In [ ]:
summer_music = zip(summer_songs, summer_artists)
winter_music = zip(winter_songs, winter_artists)
summer_lyrics = []
winter_lyrics = []
n = 0 

for song, artist in summer_music:
    try:
        url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
        words = requests.get(url).json()
        summer_lyrics.append(words)
        time.sleep(.1)
        n +=1
        print(('\n>> this is No '+ str(n) + ' in summer '))
    except: 
        summer_lyrics.append('N/A')
    
for song, artist in winter_music:
    try:
        url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
        words = requests.get(url).json()
        winter_lyrics.append(words)
        time.sleep(.1)
        n +=1
        print(('\n>> this is No '+ str(n) + ' in winter '))
        
    except:
        winter_lyrics.append('N/A')


>> this is No 1 in summer 

>> this is No 2 in summer 

>> this is No 3 in summer 

>> this is No 4 in summer 

>> this is No 5 in summer 

>> this is No 6 in summer 

>> this is No 7 in summer 

>> this is No 8 in summer 

>> this is No 9 in summer 

>> this is No 10 in summer 

>> this is No 11 in summer 

>> this is No 12 in summer 

>> this is No 13 in summer 


In [ ]:
print(winter_lyrics[0])

In [ ]:
summer_uri_list = []
winter_uri_list = []
n = 0 

for song, artist in (summer_songs, summer_artists):
    query = spotify.search('song' + 'artist')
    song_uri = query['tracks']['items'][0]['uri']
    summer_uri_list.append(song_uri)
    n +=1
    print(('\n>> this is No '+ str(n) + ' in winter '))
    time.sleep(.3)
 

In [ ]:
n = 0    
for song, artist in (winter_songs, winter_artists):
    query = spotify.search('song' + 'artist')
    song_uri = query['tracks']['items'][0]['uri']
    winter_uri_list.append(song_uri)
    time.sleep(1)
    n +=1
    
    print(('\n>> this is No '+ str(n) + ' in winter '))
    time.sleep(.3)
   

In [ ]:
# df = pd.DataFrame(list(zip(summer_songs, summer_artists, winter_songs, winter_artists)), columns = ['Summer Top Song', 
#                                                                                                     'Summer Top Artist',
#                                                                                                    'Winter Top Song',
#                                                                                                    'Winter Top Artist'])
# df